In [1]:
import os
import string
import json
import re

from utils import load_dergs
from derg import DERG, KnowledgeGraph

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("No input file specified.")? (utils.py, line 49)

In [ ]:
dergs = load_dergs('/home/liyc/data/dedroid/dergs/', 'obfuscate_derg.json')
print(len(dergs))

In [ ]:
def parse_name(name):
    """
    split an element name to words
    the name could be:
    camel case: mMyActivity, MyAwesomeClass
    underscore-splited: MY_AWESOME_FIELD, parse_names
    hybrid: create_myAwesomeClass_COMPAT
    """
    words = re.findall(r'[A-Z]?[a-z]+|[A-Z]+(?=[A-Z]|$|_)|<init>|<clinit>|\$', name)
    return map(lambda word: str.lower(str(word)), words)

print(parse_name('mMyActivity'))
print(parse_name('MyAwesomeClass'))
print(parse_name('MY_AWESOME_FIELD'))
print(parse_name('AWESOMEMethod'))
print(parse_name('parse_names'))
print(parse_name('create_myAwesomeClass_COMPAT'))
print(parse_name('create_myAwesomeClass$SubClass$access1000'))
print(parse_name('<init>'))
print(parse_name('x'))

In [ ]:
WORD_START = '<start>'
WORD_END = '<end>'
WORD_UNKNOWN = '<unknown>'

class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if word not in self.word2idx:
            return self.word2idx[WORD_UNKNOWN]
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

vocab = Vocabulary()
vocab.add_word(WORD_START)
vocab.add_word(WORD_END)
vocab.add_word(WORD_UNKNOWN)

node_to_names = {}
for derg in dergs:
    node_to_names.update(derg.get_kg_mappings())

words = []
for name in node_to_names.values():
    words.extend(parse_name(name))

from collections import Counter
counter = Counter(words)

for word in counter:
    if counter[word] > 2:
        vocab.add_word(word)

print("Total vocabulary size: %d" % len(vocab))

In [ ]:
def load_node_to_kgid_mapping(node_to_kgid_mapping_path):
    f = open(node_to_kgid_mapping_path)
    mapping = {}
    for line in f.readlines()[1:]:
        segs = line.split()
        node_global_name = segs[0]
        kgid = int(segs[1])
        mapping[node_global_name] = kgid
    return mapping

node_to_kgids = load_node_to_kgid_mapping('/home/liyc/data/dedroid/kg_without3lib/entity2id.txt')
print(len(node_to_kgids))

In [ ]:
def load_kgid_embeddings(kgid_embedding_path):
    embeddings = json.load(open(kgid_embedding_path))
    return embeddings['ent_embeddings']

kgid_embeddings = load_kgid_embeddings('/home/liyc/data/dedroid/kg_without3lib/embedding_TransE_64/embedding.vec.json')
print(len(kgid_embeddings))

In [ ]:
import random

def split_train_and_test(dergs, train_portion = 0.9):
    random.shuffle(dergs)
    sep = int(len(dergs) * train_portion)
    train_dergs = dergs[:sep]
    test_dergs = dergs[sep:]
    return train_dergs, test_dergs

train_dergs, test_dergs = split_train_and_test(dergs)
print("%d apps for training, %d apps for predicting" % (len(train_dergs), len(test_dergs)))

In [ ]:
import torch
import torch.utils.data as data
import os
import numpy as np


class DeDroidDataset(data.Dataset):
    """
    Custom Dataset compatible with torch.utils.data.DataLoader.
    """
    def __init__(self, dergs, node_to_kgids, kgid_embeddings, vocab):
        self.dergs = dergs
        self.node_to_names = []
        for g in dergs:
            self.node_to_names.extend(g.get_kg_mappings())
        self.node_to_kgids = node_to_kgids
        self.kgid_embeddings = kgid_embeddings
        self.vocab = vocab

    def __getitem__(self, index):
        """Returns one data pair (embedding and name)."""
        node_global_name, node_name = self.node_to_names[index]
        
        # Temporaral work around
        node_global_name = node_global_name[24:]
        
        kgid = self.node_to_kgids[node_global_name]
        embedding = self.kgid_embeddings[kgid]

        # Convert node name (string) to word ids.
        name_vec = []
        name_vec.append(vocab(WORD_START))
        name_vec.extend(map(vocab, parse_name(node_name)))
        name_vec.append(vocab(WORD_END))
        
        embedding = torch.Tensor(embedding)
        target = torch.Tensor(name_vec)
        return embedding, target

    def __len__(self):
        return len(self.node_to_names)


def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (embedding, name).
    
    We should build custom collate_fn rather than using default collate_fn, 
    because merging name (including padding) is not supported in default.
    Args:
        data: list of tuple (embedding, name). 
            - embedding: torch tensor of shape (embed_size).
            - name: torch tensor of shape (?); variable length.
    Returns:
        embeddings: torch tensor of shape (batch_size, embed_size).
        targets: torch tensor of shape (batch_size, padded_length).
        lengths: list; valid length for each padded name.
    """
    # Sort a data list by caption length (descending order).
    data.sort(key=lambda x: len(x[1]), reverse=True)
    embeddings, names = zip(*data)

    # Merge images (from tuple of 1D tensor to 2D tensor).
    embeddings = torch.stack(embeddings, 0)

    # Merge captions (from tuple of 1D tensor to 2D tensor).
    lengths = [len(name) for name in names]
    targets = torch.zeros(len(names), max(lengths)).long()
    for i, name in enumerate(names):
        end = lengths[i]
        targets[i, :end] = name[:end]
    return embeddings, targets, lengths


def get_loader(dergs, node_to_kgids, kgid_embeddings, vocab, batch_size, shuffle, num_workers):
    """Returns torch.utils.data.DataLoader for custom DeDroid dataset."""
    # DeDroid dataset
    dedroid_data = DeDroidDataset(dergs, node_to_kgids, kgid_embeddings, vocab)
    
    # Data loader for DeDroid dataset
    # This will return (embeddings, names, lengths) for every iteration.
    # embeddings: tensor of shape (batch_size, embed_size).
    # names: tensor of shape (batch_size, padded_length).
    # lengths: list indicating valid length for each name. length is (batch_size).
    data_loader = torch.utils.data.DataLoader(dataset=dedroid_data, 
                                              batch_size=batch_size,
                                              shuffle=shuffle,
                                              num_workers=num_workers,
                                              collate_fn=collate_fn)
    return data_loader

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence
from torch.autograd import Variable
    
    
class DeDroidRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers, dropout):
        """Set the hyper-parameters and build the layers."""
        super(DeDroidRNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.init_weights()
    
    def init_weights(self):
        """Initialize weights."""
        self.embed.weight.data.uniform_(-0.1, 0.1)
        self.linear.weight.data.uniform_(-0.1, 0.1)
        self.linear.bias.data.fill_(0)
        
    def forward(self, embeddings, names, lengths):
        """Decode kg embeddings and generate names."""
        name_embeddings = self.embed(names)
        embeddings = torch.cat((embeddings.unsqueeze(1), name_embeddings), 1)
        packed = pack_padded_sequence(embeddings, lengths, batch_first=True) 
        hiddens, _ = self.lstm(packed)
        outputs = self.linear(hiddens[0])
        return outputs
    
    def sample(self, embeddings, states=None):
        """Samples names for given kg embeddings (Greedy search)."""
        sampled_ids = []
        inputs = embeddings.unsqueeze(1)
        for i in range(10):                                      # maximum sampling length
            hiddens, states = self.lstm(inputs, states)          # (batch_size, 1, hidden_size), 
            outputs = self.linear(hiddens.squeeze(1))            # (batch_size, vocab_size)
            predicted = outputs.max(1)[1]
            sampled_ids.append(predicted)
            inputs = self.embed(predicted)
            inputs = inputs.unsqueeze(1)                         # (batch_size, 1, embed_size)
        sampled_ids = torch.stack(sampled_ids, 1)                # (batch_size, 20)
        return sampled_ids

In [ ]:
batch_size = 200
shuffle = True
num_workers = 8
embed_size = 64
hidden_size = 64
num_layers = 1
dropout = 0.2
lr = 0.006
num_epochs = 20
log_step = 500
save_step = 10000
model_path = '/home/liyc/data/dedroid/model'

def to_var(x, volatile=False):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)

def train_with_dergs(train_dergs):
    # Build data loader
    data_loader = get_loader(
        dergs = train_dergs,
        node_to_kgids = node_to_kgids,
        kgid_embeddings = kgid_embeddings,
        vocab = vocab,
        batch_size = batch_size,
        shuffle = shuffle,
        num_workers = num_workers) 

    # Build the models
    dedroid_rnn = DeDroidRNN(
        embed_size = embed_size,
        hidden_size = hidden_size,
        vocab_size = len(vocab),
        num_layers = num_layers,
        dropout = dropout
    )

    if torch.cuda.is_available():
        dedroid_rnn.cuda()

    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()
    params = list(dedroid_rnn.parameters())
    optimizer = torch.optim.Adam(params, lr=lr)

    # Train the Models
    total_step = len(data_loader)
    for epoch in range(num_epochs):
        for i, (embeddings, names, lengths) in enumerate(data_loader):
            # Set mini-batch dataset
            embeddings = to_var(embeddings, volatile=False)
            names = to_var(names)
            targets = pack_padded_sequence(names, lengths, batch_first=True)[0]

            # Forward, Backward and Optimize
            dedroid_rnn.zero_grad()
            outputs = dedroid_rnn(embeddings, names, lengths)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            # Print log info
            if i % log_step == 0:
                print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f'
                      % (epoch+1, num_epochs, i, total_step, loss.data[0], np.exp(loss.data[0]))) 

            # Save the models
            if (i+1) % save_step == 0:
                torch.save(dedroid_rnn.state_dict(), 
                           os.path.join(model_path, 'DeDroidRNN-%d-%d.pkl' % (epoch+1, i+1)))
                
    return dedroid_rnn

In [ ]:
def get_node_embedding(node_global_id):
    # Temporaral work around
    node_global_id = node_global_id[24:]
    kgid = node_to_kgids[node_global_id]
    return kgid_embeddings[kgid]
    
def predict_with_embeddings(dedroid_rnn, embeddings):
    embeddings = torch.Tensor(embeddings)
    embeddings = to_var(embeddings, volatile=True)

    # Generate names from embeddings
    sampled_ids = dedroid_rnn.sample(embeddings)
    print(sampled_ids.size())
    sampled_ids = sampled_ids.cpu().data.numpy().tolist()

    predicted_names = []
    for predicted_char_ids in sampled_ids:
        # Decode word_ids to name
        predicted_chars = []
        for word_id in predicted_char_ids:
            word = vocab.idx2word[word_id]
            if word == WORD_START or word == WORD_UNKNOWN:
                continue
            elif word == '<end>':
                break
            predicted_chars.append(word)
        predicted_name = '_'.join(predicted_chars)
        predicted_names.append(predicted_name)
    return predicted_names

def predict_random(dedroid_rnn, predict_dergs):
    # randomly pick a name to predict
    node_gid_to_names = []
    for g in predict_dergs:
        node_gid_to_names.extend(g.get_kg_mappings())
    node_gid, original_name = random.sample(node_gid_to_names, 1)[0]
    print("predicting for %s, original name is %s" % (node_gid, original_name))
    embedding = get_node_embedding(node_gid)
    predicted_name = predict_with_embeddings(dedroid_rnn, [embedding])[0]
    print("predicted name is %s" % predicted_name)

def predict_with_dergs(dedroid_rnn, predict_dergs):
    # get a list of nodes for prediction
    node_gid_to_names = []
    for g in predict_dergs:
        node_gid_to_names.extend(g.get_kg_mappings())
    node_gids, original_names = zip(*node_gid_to_names)
    print("predicting for %d nodes, original names are %s, ..." % (len(node_gids), ", ".join(original_names[:5])))
    embeddings = [get_node_embedding(node_gid) for node_gid in node_gids]
    predicted_names = predict_with_embeddings(dedroid_rnn, embeddings)
    print("predicted %d names, predicted names are %s, ..." % (len(predicted_names), ", ".join(predicted_names[:5])))
#     num_correct = sum([1 if origin == predict else 0 for origin, predict in zip(original_names, predicted_names)])
#     print("correctly predicted %d names, accuracy is %.2f." % (num_correct, float(num_correct)/len(global_names)))
    return zip(node_gids, original_names, predicted_names)

In [ ]:
import numpy as np
import itertools

def batch_train_predict(dergs, n_fold = 10):
    splited_dergs = np.array_split(dergs, n_fold)
    predictions = []
    for i in range(n_fold):
        print("Fold %d" % i)
        predict_dergs = list(splited_dergs[i])
        train_dergs = list(itertools.chain(*(splited_dergs[:i]+splited_dergs[i+1:])))
        print("%d dergs for training, %d dergs for predicting" % (len(train_dergs), len(predict_dergs)))
        dedroid_rnn = train_with_dergs(train_dergs)
        predictions.extend(predict_with_dergs(dedroid_rnn, predict_dergs))
    return predictions

predictions = batch_train_predict(dergs)
json.dump(predictions, open('/home/liyc/data/dedroid/predictions.json', 'w'), indent=2)

In [ ]:
def edit_distance(s1, s2):
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

def similarity(original_name, predicted_name):
    original_words = parse_name(original_name)
    predicted_words = parse_name(predicted_name)
    distance = edit_distance(original_words, predicted_words)
    if distance == 0:
        return 1.0
    return 1 - float(distance) / max(len(original_words), len(predicted_words))

print(similarity("Hello_world", "HelloWorldHAHA")) # should be 0.666
print(similarity("a", "b")) # should be 0
print(similarity("getData", "get")) # should be 0.5
print(similarity("", "")) # should be 1

def score(original_name, predicted_name):
    s = similarity(original_name, predicted_name)
    return s

In [ ]:
def evaluate():
    results = []
    prediction_dict = dict([(gid, predict) for gid, origin, predict in predictions])
    for derg in dergs:
        for node in derg.nodes():
            node_type = node['type']
            node_name = node['name']
            if node_type in ['class', 'method', 'field'] and len(node_name) < 3:
                original_name = node['original_name'] if 'original_name' in node else ''
                deguard_name = node['deguard_name'] if 'deguard_name' in node else ''
                if len(original_name) < 3:
                    continue
                deguard_score = score(original_name, deguard_name)
                node_gid = derg.get_node_global_id(node)
                dedroid_name = prediction_dict[node_gid] if node_gid in prediction_dict else ''
                dedroid_score = score(original_name, dedroid_name)
                results.append((node_type, original_name, node_name, deguard_name, deguard_score, dedroid_name, dedroid_score))
    return results

results = evaluate()
results

In [ ]:
print(len(results))
deguard_average_score = np.average(zip(*results)[4])
dedroid_average_score = np.average(zip(*results)[6])
print(deguard_average_score, dedroid_average_score)

In [ ]:
# Using K-means clustering to check the dataset.

import numpy as np

from sklearn.cluster import DBSCAN, KMeans
from sklearn import metrics


global_name_to_names = []
for g in dergs[:50]:
    global_name_to_names.extend(g.get_kg_mappings())
global_names, original_names = zip(*global_name_to_names)
print("using %d nodes for clustering, original names are %s, ..." % (len(global_names), ", ".join(original_names[:5])))
embeddings = [get_embedding_by_global_name(global_name) for global_name in global_names]
X = np.array(embeddings)

# #############################################################################
# Compute K-Means
kmeans = KMeans(n_clusters=1000, random_state=0).fit(X)
label_list = kmeans.labels_.tolist()

def get_label_names(label):
    for i in range(len(original_names)):
        if label_list[i] == label:
            print(original_names[i])
get_label_names(1)